In [ ]:
# old name Modulation_AllGenes_Shift
import os
import pandas as pd
import random
from Bio.Seq import MutableSeq
from Bio.Seq import Seq
from Bio.Data.CodonTable import unambiguous_dna_by_id
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from collections import Counter
from datetime import datetime

start_time = datetime.now()

total_mutations = 150*150       # ~22k достаточно для насыщения изменения кодонов
number_of_mutations = 100
number_of_generations = total_mutations // number_of_mutations     # Количество поколений


# species_list = ['Monodelphis_domestica', 'Myodes_glareolus', 'Pan_troglodytes', 'Macaca_fascicularis', 'Cercopithecus_mitis']
                                    # 'Abbottina_rivularis', 'Uroplatus_ebenaui', 'Eopsaltria_australis',
                                    # 'Tylototriton_verrucosus', 'Cavia_porcellus'

                                    # 'Monodelphis_domestica', 'Mus_musculus', 'Myodes_glareolus' ~ 5
                                    # 'Pan_troglodytes', 'Macaca_fascicularis', 'Cercopithecus_mitis' > 38
                                    # https://genomics.senescence.info/species/
                    
faggot_list = []
with open('CrossSpeciesAllGenes.txt') as cs:    # CytB - 761  AllGenes - 908
    species_list = cs.read()
species_list = species_list.split('\n')   # В файле данные вида "['species1',\n 'species2', ...]", поэтому надо убрать ',\n ' и представить в виде списка
# species_list = ASAP[100:]

codon_table = 2     # Вариации генетического кода с NCBI:
                    # 1: Standard   2: Vertebrate Mitochondrial 3: Yeast Mitochondrial
                    # 4: Mold Mitochondrial; Protozoan Mitochondrial; Coelenterate Mitochondrial; Mycoplasma; Spiroplasma
                    # 5: Invertebrate Mitochondrial     6: Ciliate Nuclear; Dasycladacean Nuclear; Hexamita Nuclear
                    # 9: Echinoderm Mitochondrial; Flatworm Mitochondrial   10: Euplotid Nuclear
                    # 11: Bacterial, Archaeal and Plant Plastid     12: Alternative Yeast Nuclear
                    # 13: Ascidian Mitochondrial    14: Alternative Flatworm Mitochondrial
                    # 16: Chlorophycean Mitochondrial   21: Trematode Mitochondrial
                    # 22: Scenedesmus obliquus Mitochondrial Code   23: Thraustochytrium mitochondrial code
                    # 24: Pterobranchia Mitochondrial   25: Candidate Division SR1; Gracilibacteria
                    # 26: Pachysolen tannophilus Nuclear Code   27: Karyorelict Nuclear     28: Condylostoma Nuclear
                    # 29: Mesodinium Nuclear    30: Peritrich Nuclear  31: Blastocrithidia Nuclear

# codons64 = sorted(list(unambiguous_dna_by_id[codon_table].forward_table.keys()) + unambiguous_dna_by_id[codon_table].stop_codons)

codons64 = ['AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC',
            'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT',
            'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC',
            'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT',
            'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT']    # Список всех возможных кодонов

for p in range(len(species_list)): # Цикл по списку видов
    start_time1 = datetime.now()
    breaker = False
    species = species_list[p] # Берет один вид из списка
    print('Расчет {} начат {}'.format(species, datetime.now()))
    MutSpecData = pd.read_csv('VertebratePolymorphisms.MutSpecData.OnlyFourFoldDegAllGenes.csv', sep=' ').copy()
    probability = 0
    mutspec = list(MutSpecData[MutSpecData.Species == species].iloc[0].values[1:])  # Вытаскивает мутспек для текущего вида
    mutspec_stack = []

    for i in range(len(mutspec)):   # Представляет мутспек в виде отрезка от 0 до 1, чтобы работала рулетка
        mutspec_stack.append(probability)
        probability += float(mutspec[i])


    for sample in range(20): # Количество генераций каждого вида(см. сколько папок уже есть в папке каждого вида)
        stage_time = datetime.now()
        # if sample % 20 == 0:  # Чекпоинты прогресса скрипта
        #     print('Цикл {} для вида {} начался  {}'.format(sample, species, datetime.now()))
        os.makedirs("Results_AllGenes_shift/{}/{}".format(species, sample), exist_ok=True)  # Создает папку для результатов

        # Генератор генома на основе CodonUsage текущего вида

        # sum_AA = pd.read_csv('CodonUsageSummary.csv', sep='\t')
        AA = pd.read_csv('AminoAcidsUsageAllGenes.csv', sep='\t').copy()
        DNA = []

        # codon_dict_AA = {
        #     'DNA': [['TTT', 'TTC'], ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
        #             ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
        #             ['TAT', 'TAC'], ['TAA', 'TAG', 'AGA', 'AGG'], ['TGT', 'TGC'], ['TGA', 'TGG'],
        #             ['CCT', 'CCC', 'CCA', 'CCG'],
        #             ['CAT', 'CAC'], ['CAA', 'CAG'], ['CGT', 'CGC', 'CGA', 'CGG'], ['ATT', 'ATC'], ['ATA', 'ATG'],
        #             ['ACT', 'ACC', 'ACA', 'ACG'], ['AAT', 'AAC'], ['AAA', 'AAG'], ['GTT', 'GTC', 'GTA', 'GTG'],
        #             ['GCT', 'GCC', 'GCA', 'GCG'], ['GAT', 'GAC'], ['GAA', 'GAG'], ['GGT', 'GGC', 'GGA', 'GGG']],
        #     'AA': ['F', 'L', 'S', 'Y', '_', 'C', 'W', 'P', 'H', 'Q', 'R', 'I', 'M', 'T', 'N', 'K', 'V', 'A', 'D', 'E',
        #            'G']}
        # amin_codon = dict(zip(codon_dict_AA['AA'], codon_dict_AA['DNA']))

        aminoacid_codon = {}     # Создает словарь {Аминокислота : [кодоны]} на основе таблиц кодонов с NCBI
        for c in unambiguous_dna_by_id[codon_table].forward_table:
            if unambiguous_dna_by_id[codon_table].forward_table[c] not in aminoacid_codon:
                aminoacid_codon[unambiguous_dna_by_id[codon_table].forward_table[c]] = []
            aminoacid_codon[unambiguous_dna_by_id[codon_table].forward_table[c]].append(c)
        aminoacid_codon['_'] = unambiguous_dna_by_id[codon_table].stop_codons

        # for l in range(10):     # Десятикратно увеличивает геном для уменьшения "стохастики" (думаю переделать:
                                # не увеличивать геном в 10 раз, а увеличить количество прогонов скрипта)
        for aminoacid in AA.columns.values[:-1]:
            odds = [0]
            for i in range(len(aminoacid_codon[aminoacid])):    # Создаем список вероятностей по вырождености аминокислоты
                odds.append(random.random() + odds[-1])
            for k in range(AA[AA.Species == species][aminoacid].values[0]):  # Цикл длиной кол-ву кодонов для аминокислоты
                odd = random.uniform(0, odds[-1])
                for q in range(len(aminoacid_codon[aminoacid])):    # Для учета вырожденности АА
                    if odd < odds[q+1] and odd > odds[q]:
                        DNA.append(aminoacid_codon[aminoacid][q])

        CodonUsage = pd.DataFrame(columns=codons64)     # Создаем датафреймы для подсчета CodonUsage и
        Frequency = pd.DataFrame(columns=codons64)
        observed_frequency = pd.read_csv('CodonFrequencyAllGenes.csv', sep='\t').copy()
        Frequency.loc[0] = list(observed_frequency[observed_frequency.Species == species].values[0][1:])
            # Записывает частоты кодонов из живого генома текущего вида

        mutations = ['A_T', 'A_G', 'A_C', 'T_A', 'T_G', 'T_C', 'G_A', 'G_T', 'G_C', 'C_A', 'C_T', 'C_G']
            # Список однонуклеотидных замен

        for generation in range(number_of_generations):
            faggot_counter = 0
            if generation % 1 == 0: # Регулирует раз в сколько поколений считать частоты кодонов
                # Разбивка на список кодонов
                # codons_of_seq = []
                # for l in range(len(seq)//3):
                #     codons_of_seq.append(seq.toseq()[l*3:l*3+3])
                # print('Время разбивки: {}'.format(datetime.now( ) - stage_time))
                codon_usage = Counter(DNA)      # Расчет CodonUsage
                if len(list(codon_usage)) < 64:                     # Проверка наличия всех 64 кодонов
                    for t in codons64:                              # без нее возникали ошибки
                        if t not in list(codon_usage):              # если кодон исчезает приписываем ему ноль
                            codon_usage[t] = 0
                for i in range(64):     # Записывает CodonUsage
                    CodonUsage.at[generation, CodonUsage.columns[i]] = codon_usage[CodonUsage.columns[i]]
                frequency_values = []  # Список значений частот кодонов
                total = len(DNA)   # Сумма всех кодонов
                for e in range(64):
                    frequency_values.append(CodonUsage.loc[generation].values[e] / total)   # Расчитывает частоту каждого кодона
                Frequency.loc[generation + 1] = frequency_values  # Записывает в строку текущего поколения
                CodonUsage.to_csv('Results_AllGenes_shift\{}\{}\Codons.csv'.format(species, sample), sep='\t', index=False)
                Frequency.to_csv('Results_AllGenes_shift\{}\{}\Frequencies.csv'.format(species, sample), sep ='\t', index=False)
                # Сохраняет значения

            # Мутагенез через строку нуклеотидов
            # for codon_id in range(len(seq) // 3):
            #     codon = seq[codon_id*3:codon_id*3+3]
            #     mut_id = random.randint(0, 2)
            #     mut_probability = random.uniform(0, mutspec_norm[-1])
            #     counter = 0
            #     while mut_probability >= mutspec_norm[counter + 1]:
            #         counter += 1
            #     if codon[mut_id] == mutations[counter][0]:
            #         new_codon = codon
            #         codon = codon.toseq()
            #         new_codon[mut_id] = mutations[counter][2]
            #         if codon.translate(table="Vertebrate Mitochondrial") == new_codon.toseq().translate(
            #                 table="Vertebrate Mitochondrial"):
            #             seq[codon_id*3:codon_id*3+3] = new_codon


            # Мутагенез через список кодонов
            mut_counter = 0

            while mut_counter < number_of_mutations:        # Цикл по всей ДНК # and generation < number_of_generations-1
                codon_id = random.randint(0, len(DNA)-1)
                codon = MutableSeq(DNA[codon_id])   # Выбираем отдельный кодон
                mut_id = random.randint(0, 2)       # Выбираем нуклеотид из этого кодона
                mut_probability = random.uniform(0, mutspec_stack[-1])  # Задаем значение рулетки
                counter = 0
                while mut_probability >= mutspec_stack[counter + 1]:    # Смотрим на какою именно замену она выпала
                    counter += 1
                if codon[mut_id] == mutations[counter][0]:              # Сопоставляем нуклеотид замены из мутспека с
                    new_codon = codon                                   # выбранным нуклеотидом из кодона
                    codon = Seq(codon)
                    new_codon[mut_id] = mutations[counter][2]           # Подставляем нуклеотид в кодоне

                    if codon.translate(table=codon_table) == Seq(new_codon).translate(table=codon_table):
                        DNA[codon_id] = str(new_codon)     # Проверка на синонимичность мутировавшего кодона
                        mut_counter += 1
                faggot_counter += 1
                if faggot_counter > 15000:
                    faggot_list.append(species)
                    print('{} is a faggot'.format(species))
                    breaker = True
                    break
            if breaker == True:
                break
            # print('Время мутагенеза: {}'.format(datetime.now( ) - stage_time))
            # stage_time = datetime.now()
        if breaker == True:
            break


        # Расчет корреляций и p-value
        data = pd.read_csv('Results_AllGenes_shift\{}\{}\Frequencies.csv'.format(species, sample), sep='\t').copy()
        spearman = []
        pearson = []
        pv_s = []
        pv_p = []
        for i in data.index.values[1:]:
            data1, data2 = data.loc[0], data.loc[i]
            corrp, p_p = pearsonr(data1, data2)
            corrs, p_s = spearmanr(data1, data2)
            spearman.append(corrs)
            pearson.append(corrp)
            pv_s.append(p_s)
            pv_p.append(p_p)
        spear_corr = pd.DataFrame(index=[i for i in range(number_of_generations)], columns=['spearman', 'pv_s', 'pearson', 'pv_p'])
        spear_corr['spearman'] = spearman
        spear_corr['pearson'] = pearson
        spear_corr['pv_s'] = pv_s
        spear_corr['pv_p'] = pv_p
        spear_corr.to_csv('Results_AllGenes_shift\{}\{}\Spearman.csv'.format(species, sample), sep='\t', index=False)    # Расчет корреляций закончен

    if breaker == True:
        continue


    # Расчет средних частот кодонов для ВСЕХ смодулированных образцов и корреляций
    samples_number = 0
    for i in next(os.walk('Results_AllGenes_shift\{}'.format(species)))[1]:     # Количество смодулированных образцов
        if i.isnumeric(): samples_number += 1                           # os.walk выдает кортежи(path, dirnames, filenames)

    # Средние частоты
    FrequenciesMean = pd.read_csv('Results_AllGenes_shift\{}\{}\Frequencies.csv'.format(species, 0), sep='\t').copy()
        # Записывает частоты живого генома текущего вида в датафрейм текущего вида

    for sample in range(1, samples_number):
        freq = pd.read_csv('Results_AllGenes_shift\{}\{}\Frequencies.csv'.format(species, sample), sep='\t').copy()
        for i in codons64:
            FrequenciesMean[i] += freq[i]
    for i in codons64:
        FrequenciesMean[i] = FrequenciesMean[i] / samples_number
    FrequenciesMean.to_csv('Results_AllGenes_shift\{}\FrequenciesMean.csv'.format(species), sep='\t', index=False)

    # Корреляции
    data = pd.read_csv('Results_AllGenes_shift\{}\FrequenciesMean.csv'.format(species), sep='\t').copy()
    spear_corr = pd.DataFrame(index=[i for i in range(number_of_generations)])
    spearman = []
    pearson = []
    pv_s = []
    pv_p = []
    for i in data.index.values[1:]:
        data1, data2 = data.loc[0], data.loc[i]
        corrp, p_p = pearsonr(data1, data2)
        corrs, p_s = spearmanr(data1, data2)
        spearman.append(corrs)
        pearson.append(corrp)
        pv_s.append(p_s)
        pv_p.append(p_p)
    spear_corr['spearman'] = spearman
    spear_corr['pearson'] = pearson
    spear_corr['pv_s'] = pv_s
    spear_corr['pv_p'] = pv_p
    spear_corr.to_csv('Results_AllGenes_shift\{}\SpearmanMean.csv'.format(species), sep='\t', index=False)

    # print('Расчет средних корреляций закончен')

    # print('Расчет {} закончен   {}'.format(species, datetime.now()))
    print(datetime.now() - start_time1)
print(datetime.now() - start_time)
print(faggot_list)

In [ ]:
# Негативный контроль рыб(равномерный мутспек)

# Замененный мутспеки у тепло- и холоднокровных
# warm_blooded = ['Agelaius_phoeniceus', 'Alauda_arvensis', 'Alectoris_chukar', 'Anas_platyrhynchos', 'Anser_cygnoides', 'Bambusicola_thoracicus', 'Brotogeris_cyanoptera', 'Cardinalis_cardinalis', 'Carpodacus_erythrinus', 'Copsychus_saularis', 'Cyanopica_cyanus', 'Eopsaltria_australis', 'Eopsaltria_griseogularis', 'Fringilla_coelebs', 'Gallus_gallus', 'Garrulax_canorus', 'Garrulus_glandarius', 'Grus_canadensis', 'Henicorhina_leucosticta', 'Acomys_cahirinus', 'Ailuropoda_melanoleuca', 'Alouatta_seniculus', 'Anourosorex_squamipes', 'Aotus_nancymaae', 'Apodemus_draco', 'Apodemus_latronum', 'Apodemus_peninsulae', 'Arctocephalus_forsteri', 'Artibeus_jamaicensis', 'Artibeus_lituratus', 'Ateles_belzebuth', 'Avahi_laniger', 'Balaena_mysticetus', 'Bandicota_indica', 'Bos_indicus', 'Bos_javanicus', 'Bos_taurus', 'Bubalus_bubalis', 'Callosciurus_erythraeus', 'Canis_lupus', 'Canis_lupus_familiaris', 'Capra_aegagrus', 'Capreolus_capreolus', 'Capreolus_pygargus', 'Carollia_perspicillata', 'Cavia_porcellus', 'Cercopithecus_albogularis', 'Cercopithecus_mitis', 'Cervus_elaphus', 'Cheirogaleus_medius', 'Chlorocebus_cynosuros', 'Chlorocebus_pygerythrus', 'Chlorocebus_tantalus', 'Colobus_guereza', 'Cricetulus_kamensis', 'Crocidura_attenuata', 'Crocidura_beatus', 'Crocidura_grayi', 'Crocidura_mindorus', 'Crocidura_russula', 'Crocidura_shantungensis', 'Cynomys_ludovicianus', 'Cynopterus_brachyotis', 'Dasypus_novemcinctus', 'Dasyurus_hallucatus', 'Daubentonia_madagascariensis', 'Desmodus_rotundus', 'Dremomys_rufigenis', 'Eospalax_baileyi', 'Eospalax_cansus', 'Eothenomys_chinensis', 'Eothenomys_melanogaster', 'Eothenomys_miletus', 'Eozapus_setchuanus', 'Episoriculus_caudatus', 'Episoriculus_fumidus', 'Episoriculus_macrurus', 'Epomophorus_gambianus', 'Equus_asinus', 'Equus_caballus', 'Euchoreutes_naso', 'Gazella_subgutturosa', 'Gorilla_gorilla', 'Gorilla_gorilla_gorilla', 'Herpestes_javanicus', 'Heterocephalus_glaber', 'Hipposideros_armiger', 'Hylobates_agilis', 'Hylobates_lar', 'Hylomys_suillus', 'Indri_indri', 'Lama_guanicoe', 'Laonastes_aenigmamus', 'Lasiurus_borealis', 'Leopoldamys_edwardsi', 'Lepilemur_ruficaudatus', 'Lepus_americanus', 'Lepus_capensis', 'Lepus_europaeus', 'Lepus_sinensis', 'Lepus_timidus', 'Lophostoma_silvicolum', 'Loris_tardigradus', 'Macaca_fascicularis', 'Macaca_mulatta', 'Macaca_nemestrina', 'Macaca_tonkeana', 'Martes_flavigula', 'Martes_zibellina', 'Meles_meles', 'Meriones_libycus', 'Meriones_meridianus', 'Microcebus_murinus', 'Micromys_minutus', 'Micronycteris_megalotis', 'Mogera_wogura', 'Monodelphis_domestica', 'Mustela_frenata', 'Mustela_nivalis', 'Mustela_sibirica', 'Mus_caroli', 'Mus_musculus', 'Myodes_glareolus', 'Myodes_rufocanus', 'Myotis_davidii', 'Myotis_lucifugus', 'Myotis_muricola', 'Nasua_nasua', 'Neodon_irene', 'Neomys_fodiens', 'Neotetracus_sinensis', 'Niviventer_confucianus', 'Niviventer_excelsior', 'Niviventer_fulvescens', 'Nyctalus_noctula', 'Nycticebus_coucang', 'Ochotona_collaris', 'Ochotona_curzoniae', 'Ochotona_princeps', 'Ornithorhynchus_anatinus', 'Ovis_ammon', 'Pan_troglodytes', 'Papio_anubis', 'Papio_cynocephalus', 'Pecari_tajacu', 'Perameles_gunnii', 'Perodicticus_potto', 'Petaurista_alborufus', 'Plecotus_auritus', 'Plecotus_macrobullaris', 'Potorous_tridactylus', 'Presbytis_melalophos', 'Prionailurus_viverrinus', 'Procyon_lotor', 'Propithecus_diadema', 'Propithecus_edwardsi', 'Propithecus_verreauxi', 'Pseudois_nayaur', 'Pteromys_volans', 'Pteronotus_parnellii', 'Rangifer_tarandus', 'Rattus_rattus', 'Rattus_tanezumi', 'Rattus_tiomanicus', 'Rhinolophus_macrotis', 'Rhinopithecus_bieti', 'Rousettus_aegyptiacus', 'Semnopithecus_entellus', 'Sorex_araneus', 'Sorex_cylindricauda', 'Sorex_unguiculatus', 'Suncus_murinus', 'Sus_celebensis', 'Sus_scrofa', 'Symphalangus_syndactylus', 'Tamias_sibiricus', 'Tamiops_maritimus', 'Tamiops_swinhoei', 'Thylamys_elegans', 'Trachypithecus_francoisi', 'Tragelaphus_scriptus', 'Tupaia_belangeri', 'Tursiops_aduncus', 'Tursiops_truncatus', 'Urocyon_cinereoargenteus', 'Urotrichus_talpoides', 'Ursus_arctos', 'Ursus_thibetanus', 'Vicugna_vicugna', 'Viverricula_indica', 'Vulpes_vulpes']
# cold_blooded = ['Abbottina_rivularis', 'Abudefduf_vaigiensis', 'Acanthogobius_hasta', 'Acanthopagrus_schlegelii', 'Acanthorhodeus_chankaensis', 'Acheilognathus_barbatus', 'Acheilognathus_macropterus', 'Acheilognathus_signifer', 'Acheilognathus_somjinensis', 'Acipenser_gueldenstaedtii', 'Acipenser_schrenckii', 'Acrossocheilus_parallens', 'Acrossocheilus_yunnanensis', 'Agonostomus_monticola', 'Alburnus_alburnus', 'Amphiprion_clarkii', 'Amphiprion_ocellaris', 'Amphiprion_percula', 'Anabas_testudineus', 'Anguilla_anguilla', 'Anguilla_bicolor_bicolor', 'Anguilla_marmorata', 'Apteronotus_albifrons', 'Auxis_rochei', 'Auxis_thazard', 'Barbatula_barbatula', 'Barbatula_nuda', 'Barbatula_toni', 'Beaufortia_kweichowensis', 'Beryx_splendens', 'Biwia_zezera', 'Boleophthalmus_pectinirostris', 'Brachyhypopomus_occidentalis', 'Brachymystax_lenok', 'Campostoma_anomalum', 'Carassius_auratus', 'Carassius_auratus_auratus', 'Carassius_carassius', 'Carassius_cuvieri', 'Carassius_gibelio', 'Catla_catla', 'Channa_argus', 'Channa_asiatica', 'Channa_maculata', 'Chuanchia_labiosa', 'Cirrhinus_mrigala', 'Clarias_gariepinus', 'Cleisthenes_herzensteini', 'Clupea_harengus', 'Clupea_pallasii', 'Cobitis_biwae', 'Cobitis_sinensis', 'Cobitis_striata', 'Coilia_mystus', 'Colomesus_asellus', 'Coregonus_lavaretus', 'Coregonus_muksun', 'Coregonus_oxyrinchus', 'Coreoleuciscus_splendidus', 'Coreoperca_kawamebari', 'Cottus_bairdii', 'Ctenopharyngodon_idella', 'Culter_alburnus', 'Cyprinella_lutrensis', 'Cyprinodon_rubrofluviatilis', 'Cyprinodon_variegatus', 'Cyprinus_carpio', 'Cyprinus_multitaeniata', 'Danio_rerio', 'Dicentrarchus_labrax', 'Dicentrarchus_punctatus', 'Diplomystes_nahuelbutaensis', 'Diptychus_maculatus', 'Elassoma_evergladei', 'Elassoma_zonatum', 'Eleutheronema_tetradactylum', 'Elopichthys_bambusa', 'Engraulis_encrasicolus', 'Epinephelus_akaara', 'Epinephelus_fuscoguttatus', 'Epinephelus_lanceolatus', 'Erpetoichthys_calabaricus', 'Esox_lucius', 'Euchiloglanis_kishinouyei', 'Exocoetus_volitans', 'Fundulus_heteroclitus', 'Fundulus_notatus', 'Fundulus_olivaceus', 'Gadus_chalcogrammus', 'Gadus_morhua', 'Galaxias_maculatus', 'Galaxias_truttaceus', 'Garra_orientalis', 'Glyptothorax_zanaensis', 'Gobiobotia_brevibarba', 'Gobiobotia_macrocephala', 'Gobiobotia_naktongensis', 'Gobio_gobio', 'Gymnocypris_eckloni', 'Gymnocypris_przewalskii', 'Gymnodiptychus_dybowskii', 'Gymnodiptychus_pachycheilus', 'Hemibagrus_guttatus', 'Hemibagrus_macropterus', 'Heteropneustes_fossilis', 'Hippocampus_ingens', 'Hypophthalmichthys_molitrix', 'Hypophthalmichthys_nobilis', 'Katsuwonus_pelamis', 'Larimichthys_polyactis', 'Lepomis_macrochirus', 'Lota_lota', 'Lutjanus_argentimaculatus', 'Micromesistius_poutassou', 'Micropterus_dolomieu', 'Micropterus_salmoides', 'Mugil_cephalus', 'Mugil_curema', 'Oncorhynchus_keta', 'Oncorhynchus_mykiss', 'Oreochromis_niloticus', 'Oryzias_latipes', 'Osmerus_mordax', 'Pampus_argenteus', 'Panthera_leo', 'Paralichthys_olivaceus', 'Pellonula_leonensis', 'Perca_fluviatilis', 'Phoxinus_phoxinus', 'Platichthys_stellatus', 'Plectropomus_leopardus', 'Plotosus_lineatus', 'Poecilia_reticulata', 'Pollachius_virens', 'Pseudopleuronectes_yokohamae', 'Pungitius_pungitius', 'Rastrelliger_kanagurta', 'Reinhardtius_hippoglossoides', 'Salmo_salar', 'Salmo_trutta', 'Scomberomorus_munroi', 'Scomberomorus_semifasciatus', 'Scomber_japonicus', 'Seriola_dumerili', 'Siganus_canaliculatus', 'Spicara_maena', 'Sprattus_sprattus', 'Squaliobarbus_curriculus', 'Takifugu_rubripes', 'Terapon_jarbua', 'Thunnus_alalunga', 'Thunnus_thynnus', 'Tropheus_moorii', 'Xiphias_gladius', 'Zeus_faber', 'Andrias_davidianus', 'Batrachuperus_pinchonii', 'Bombina_maxima', 'Bombina_variegata', 'Desmognathus_fuscus', 'Desmognathus_wrighti', 'Duttaphrynus_melanostictus', 'Echinotriton_andersoni', 'Gyrinophilus_porphyriticus']